# JIGSAW DEMONSTRATION
Demonstrates Jigsaw working, and breaking Jigsaw through it's renormalisation

In [ ]:
import qiskit
import numpy as np
import copy
import PatchedMeasCal.jigsaw as jigsaw
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal import bv, qft
from PatchedMeasCal.fake_backends import Grid, Hexagonal, Hexagonal16
from PatchedMeasCal.utils import norm_results_dict

from PatchedMeasCal.state_prep_circuits import integer_state_prep_sim, integer_state_dist

from PatchedMeasCal.gen_error_matrices import FakeMeasurementError

from PatchedMeasCal.fake_backends import LocalSimulator


import qiskit.tools.jupyter

In [ ]:
n_qubits = 4
n_shots = 16000
bv_str = '0' * (n_qubits - 2) + '11'
targ_bv_str = bv_str[::-1]

backend = Grid(2, 2) #Hexagonal16() #Grid(3, 3)

circuit = bv.bv_circuit(bv_str, n_qubits)
print(targ_bv_str)
backend

## Bare execution

In [ ]:
initial_layout = list(range(n_qubits))

tc = qiskit.transpile(circuit, backend=backend, optimization_level=0, initial_layout=initial_layout)
res_d = qiskit.execute(tc, backend, shots=n_shots, optimization_level=0, initial_layout=initial_layout).result().get_counts()
res_bare = copy.deepcopy(res_d)
norm_results_dict(res_d)
print(res_d[targ_bv_str])

## Run Jigsaw

In [ ]:
res_j = jigsaw.jigsaw(circuit, backend, n_shots, verbose=True, local_pmf_pairs = [(0, 3), (1, 2)])
print(res_j[targ_bv_str])

## Try CMC/TPF

In [ ]:
tpf = TensorPatchFitter(backend, n_shots=n_shots)
tpf.build()

In [ ]:
t_res = tpf.apply(res_d)
print(t_res[targ_bv_str])

## What if circuit filter + backend filter?

In [ ]:
tj_res = tpf(res_j)
print(tj_res[targ_bv_str])

In [ ]:
tpf_filter = FakeMeasurementError(n_qubits=4, meas_filter=tpf)
jt_res = jigsaw.jigsaw(circuit, backend, n_shots, verbose=True, local_pmf_pairs = [(0, 3), (1, 2)], probs=tpf_filter)
print(res_j[targ_bv_str])

## Breaking JIGSAW via Normalisation


In [ ]:
n_qubits=4
circuit = integer_state_prep_sim(2, n_qubits)
circuit.draw()

# Target state is '0100' as Qiskit reports measured qubits backwards

In [ ]:
n_qubits = 4

aligned_cmap = [[0, 3], [2, 1]]
anti_aligned_cmap = [[0, 1], [2, 3]]

backend = LocalSimulator(n_qubits)
probs = FakeMeasurementError(
        [100, 0, 10,0,0], # Const - Controls correlation of error weights
        [0, 0 , 0, 0, 0], # 1 -> 0 - Controls error biases
        [0, 0, 0, 0, 0], # 0 -> 1 - Controls error biases
        n_qubits=n_qubits,
        coupling_map = aligned_cmap,
        norm_error=0.75,
        )

### Aligned Jigsaw

In [ ]:
# jigsaw pairs align with the errors
jigsaw.jigsaw(circuit, backend, 
              100, n_qubits=n_qubits, probs=probs, 
              local_pmf_pairs=aligned_cmap)

### Anti-Aligned Jigsaw

In [ ]:
# jigsaw pairs anti_align with the errors
jigsaw.jigsaw(circuit, backend, 
              100, n_qubits=n_qubits, probs=probs, 
              local_pmf_pairs=anti_aligned_cmap)

### Aligned CMC

In [ ]:
res_d = probs(qiskit.execute(circuit, backend, shots=16000, optimization_level=0, initial_layout=initial_layout).result().get_counts())
tpf = TensorPatchFitter(backend, n_shots=16000, coupling_map=aligned_cmap)
tpf.build(probs=probs)
print('Bare:', res_d)
print('TPF:', tpf.apply(tpf.apply(res_d)))

### Anti-aligned CMC

In [ ]:
# CMC anti-aligned -> I
res_d = probs(qiskit.execute(circuit, backend, shots=16000, optimization_level=0, initial_layout=initial_layout).result().get_counts())
tpf = TensorPatchFitter(backend, n_shots=16000, coupling_map=anti_aligned_cmap)
tpf.build(probs=probs)
print('Bare:', res_d)
print('TPF:', tpf.apply(tpf.apply(res_d)))